In [116]:
import numpy as np
import pandas as pd
import os
import math
os.getcwd()

'C:\\Users\\prje\\OneDrive - SUPERETTE\\Bureau\\IODAA\\APT\\Fil Rouge\\data_CROUS\\data'

In [109]:
#Preprocess manuel des fichiers avant import : 
#- Uniformasation des noms de colonnes (Retrait des espaces en trop dans certains noms + majuscule en trop dans "Seule ou en groupe" dans un des csv)
#- Remplacement des valeur noté manuellement N.A par des espaces --> Permet à pd.dropna de bien fonctionner
#- Retrait de valeur avec "?" dans "Nb convives" --> Remplacement par la valeur supposée
#- Retrait de valeur sous les tableaux (calcul et N° de formulaire sans valeurs dans leurs lignes) / A côté quand le nombre d'amis dépasse les colonnes 
#- Coquille dans formulaire 28-oct ligne 53 colonne taille 157 --> 1,57
csv_1 = pd.read_csv("Resultats_formulaires_21_octobre.csv", sep=";")
csv_2 = pd.read_csv("Resultats_formulaires_22_octobre.csv", sep=";")
csv_3 = pd.read_csv("Resultats_formulaires_28_octobre.csv", sep=";")
csv_4 = pd.read_csv("Resultats_formulaires_29_octobre.csv", sep=";")
csv_tot = pd.concat([csv_1, csv_2,csv_3, csv_4])
csv_tot = csv_tot.set_index("N° Formulaire")

In [110]:

csv_light = csv_tot.drop(columns=["Ami 1", "Ami 2","Ami 3","Ami 4","Ami 5",'Unnamed: 28','Unnamed: 27','Unnamed: 29'])
csv_light.columns


Index(['Genre', 'Âge', 'Taille (m)', 'Poids (kg)', 'IMC', 'Activité physique',
       'Régime particulier', 'Végé', 'Fréquence CROUS', 'Equilibre alim',
       'Seule ou en groupe', 'Nb groupe entrée', 'Proximité', 'Connaissance',
       'Parti groupe', 'Influ sociale', 'Percept senso', 'Nb file', 'Faim',
       'Stress', 'Nb convives'],
      dtype='object')

In [134]:
csv_light_wo_na = csv_light.dropna(axis=0, how="any")
csv_light_wo_na

,Genre,Âge,Taille (m),Poids (kg),IMC,Activité physique,Régime particulier,Végé,Fréquence CROUS,Equilibre alim,...,Nb groupe entrée,Proximité,Connaissance,Parti groupe,Influ sociale,Percept senso,Nb file,Faim,Stress,Nb convives
N° Formulaire,,,,,,,,,,,,,,,,,,,,,
5I3E,M,21,"1,78",66,"20,83070319",Modérée,Non,Non,Tous les jours ou presque,Moyennement,...,4.0,Oui,Non,Non,Pas influencé,Très influencé,"Non,pas trop","Oui, très faim",Plutôt oui,4.0
9J6M,M,21,"1,85",68,"19,86851717",Faible,Oui,Oui,Tous les jours ou presque,Moyennement,...,4.0,Oui,Oui,Oui,Moyennement influencé,Moyennement influencé,Plutôt oui,"Oui, très faim",Plutôt non,4.0
5B2Z,M,21,"1,95",68,"17,88297173",Modérée,Non,Non,Tous les jours ou presque,Un peu,...,4.0,Oui,Oui,Oui,Pas influencé,Pas influencé,"Non, je ne pense pas","Oui, très faim",Non,4.0
0O5Q,M,21,"1,7",70,"24,22145329",Faible,Non,Non,Tous les jours ou presque,Un peu,...,4.0,Non,Non,Oui,Pas influencé,Peu influencé,"Non, je ne pense pas","Oui, assez faim faim",Non,4.0
9P8X,M,22,"1,7",70,"24,22145329",Importante,Non,Non,Au moins 1 fois par semaine,Très attention,...,3.0,Non,Oui,Oui,Peu influencé,Pas influencé,Plutôt oui,"Oui, assez faim faim",Plutôt non,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0W3D,F,24,"1,67",62,"22,23098713",Modérée,Oui,Oui,Je viens pour la première fois,Un peu,...,1.0,Oui,Oui,Non,Peu influencé,Peu influencé,Plutôt oui,"Oui, mais très peu","Oui, très",1.0
1U7O,M,20,"1,74",57,"18,8267935",Modérée,Non,Non,Au moins 1 fois par semaine,Moyennement,...,10.0,Oui,Non,Non,Peu influencé,Moyennement influencé,Plutôt oui,"Oui, très faim",Plutôt oui,3.0
2Z9A,M,19,"1,82",70,"21,13271344",Importante,Non,Non,Au moins 1 fois par semaine,Moyennement,...,10.0,Oui,Oui,Oui,Peu influencé,Moyennement influencé,"Non, pas trop","Oui, très faim",Non,3.0


In [135]:
#Réduction en classes des colonnes avec des valeurs très diverses pour allégre le one-hot encoding
def round_to_nearest_IMC(x) : 
    if int(x.split(",")[1][0]) > 5 : 
        return int(x.split(",")[0]) + 1
    else : 
        return int(x.split(",")[0])
    
def round_to_nearest_Height(x) : 
    #Arrondi par classe de 10 cms
    try : 
        meter = x.split(",")[0]
        centimeter = x.split(",")[1][0]
        return (float(meter + "." + centimeter))
    except : 
        print(x)

def round_to_nearest_Weight(x) : 
    #Arrondi par classe de 5 kg
    if type(x) == int :
        weight = x - x%5
        return (weight)
    else : 
        x = float(x.replace(",","."))
        weight = x - x%5
        return (weight)
        
        
csv_light_wo_na["IMC"] = csv_light_wo_na["IMC"].map(lambda IMC : round_to_nearest_IMC(IMC))
print(csv_light_wo_na["IMC"])

csv_light_wo_na["Taille (m)"] = csv_light_wo_na["Taille (m)"].map(lambda Taille : round_to_nearest_Height(Taille))
print(csv_light_wo_na["Taille (m)"])
            
csv_light_wo_na["Poids (kg)"] = csv_light_wo_na["Poids (kg)"].map(lambda Poids : round_to_nearest_Weight(Poids))
print(csv_light_wo_na["Poids (kg)"])

N° Formulaire
5I3E    21
9J6M    20
5B2Z    18
0O5Q    24
9P8X    24
        ..
0W3D    22
1U7O    19
2Z9A    21
7B7K    32
6X9F    24
Name: IMC, Length: 377, dtype: int64
157
N° Formulaire
5I3E    1.7
9J6M    1.8
5B2Z    1.9
0O5Q    1.7
9P8X    1.7
       ... 
0W3D    1.6
1U7O    1.7
2Z9A    1.8
7B7K    1.5
6X9F    1.7
Name: Taille (m), Length: 377, dtype: float64
N° Formulaire
5I3E    65.0
9J6M    65.0
5B2Z    65.0
0O5Q    70.0
9P8X    70.0
        ... 
0W3D    60.0
1U7O    55.0
2Z9A    70.0
7B7K    75.0
6X9F    75.0
Name: Poids (kg), Length: 377, dtype: float64


C:\Users\prje\AppData\Local\Temp\ipykernel_19344\260233342.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_light_wo_na["IMC"] = csv_light_wo_na["IMC"].map(lambda IMC : round_to_nearest_IMC(IMC))
C:\Users\prje\AppData\Local\Temp\ipykernel_19344\260233342.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_light_wo_na["Taille (m)"] = csv_light_wo_na["Taille (m)"].map(lambda Taille : round_to_nearest_Height(Taille))
C:\Users\prje\AppData\Local\Temp\ipykernel_19344\260233342.py:34: SettingWithCopy

In [136]:
#On veut garder les N° de formulaire comme index
#Sortie : 377*72
one_hot = pd.get_dummies(csv_light_wo_na)
one_hot


,Âge,Taille (m),Poids (kg),IMC,Nb groupe entrée,Nb convives,Genre_F,Genre_M,Activité physique_Aucune,Activité physique_Aucune,...,"Faim_Oui, assez faim","Faim_Oui, assez faim faim","Faim_Oui, mais très peu","Faim_Oui, très faim","Faim_Oui,très faim",Stress_Non,"Stress_Oui, très","Stress_Oui, très",Stress_Plutôt non,Stress_Plutôt oui
N° Formulaire,,,,,,,,,,,,,,,,,,,,,
5I3E,21,1.7,65.0,21,4.0,4.0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
9J6M,21,1.8,65.0,20,4.0,4.0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
5B2Z,21,1.9,65.0,18,4.0,4.0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
0O5Q,21,1.7,70.0,24,4.0,4.0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
9P8X,22,1.7,70.0,24,3.0,2.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0W3D,24,1.6,60.0,22,1.0,1.0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1U7O,20,1.7,55.0,19,10.0,3.0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
2Z9A,19,1.8,70.0,21,10.0,3.0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [140]:
one_hot.to_csv("One_Hot_Formulaires.csv", encoding='latin1')